<a href="https://colab.research.google.com/github/aubricot/computer_vision_with_eol_images/blob/master/classification_for_image_tagging/image_type/cartoonify_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Determine if images are a cartoon or photograph
---
*Last Updated 30 October  2020*   
Cartoonify image, then compare change in color values. If change above a certain threshold, then image is likely a photograph. If change below a certain threshold, image is likely a cartoon.

**Results**   
Using 500 images from all image type classes, the best predictor of "not cartoon" was found to be Manhattan norm per pixel > 2.

### Imports
---

In [ ]:
# Mount google drive to import/export files
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# For saving images
!pip install pillow
!pip install scipy==1.1.0
import scipy

# For importing data and images
import pandas as pd
import numpy as np
import os
from scipy.linalg import norm
from scipy import sum, average

# For saving images
from scipy import misc

# For drawing on and displaying images
import cv2
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'svg'

%cd /content/drive/My Drive/summer20/classification/image_type/images

### Cartoonization
---

In [ ]:
from PIL import Image
import time

# TO DO: Select image folder to use for determining "cartoon threshold"
imclass = "phylo" #@param ["illus", "phylo", "herb", "maps", "null"]
image_paths = os.listdir(imclass)
path = imclass + "/"
test_paths = [os.path.join(path, image_path) for image_path in image_paths]

fns = []
mnorms = []
mnorms_pp = []
znorms = []
znorms_pp = []

# Cartoonify images
for i, fn in enumerate(test_paths[1:50], start=1):
  start = time.time()
  # Read in image
  img = cv2.imread(fn)
  # Display image
  #_, ax = plt.subplots(figsize=(10, 10))
  #plt.title("Original")
  #ax.imshow(img)
  # Make edges 
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
  gray = cv2.medianBlur(gray, 5) 
  edges = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C,  
                                         cv2.THRESH_BINARY, 9, 9)  
  edges = cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)
  # Cartoonization - bilateral filter and edges 
  color = cv2.bilateralFilter(img, 9, 250, 250) 
  img2 = cv2.bitwise_and(color, edges) 

  # Calculate difference in original and cartoonized image
  # Convert both images from RGB to HSV
  HSV_img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
  HSV_img2 = cv2.cvtColor(img2, cv2.COLOR_RGB2HSV)
  # Fnd the difference for H of HSV values of the images
  diff = HSV_img[:,:,0]-HSV_img2[:,:,0]
  # Cartoon: 148201490, 1202978; 6, 0.0487
  # Image: 4229132, 378998; 18.7, 0.16
  mnorm = sum(abs(diff))  # Manhattan norm
  mnorm_pp = mnorm/HSV_img.size # per pixel
  znorm = norm(diff.ravel(), 0)  # Zero norm
  znorm_pp = znorm*1.0/HSV_img2.size # per pixel
  end = time.time()

  # Display cartoonized image
  #_, ax = plt.subplots(figsize=(10, 10))
  #plt.title("Cartoonized \n Manhattan norm: {} / per pixel {} \
  #\n Zero norm: {} / per pixel {}".format(mnorm, mnorm_pp, znorm, znorm_pp))
  #ax.imshow(img2)
  # Save cartoonized image
  #savepath = os.path.splitext(impath)[0] + "_cartoon.jpg"
  #misc.imsave(savepath, img2)

  # For exporting results
  fns.append(fn)
  print("Completed for {}, {} of {} files in {} seconds".format(fn, i, len(image_paths), format(end-start, '.2f')))
  mnorms.append(mnorm)
  mnorms_pp.append(mnorm_pp)
  znorms.append(znorm)
  znorms_pp.append(znorm_pp)

# Combine to df
cartoonif_vals = pd.DataFrame(([fns, mnorms, mnorms_pp, znorms, znorms_pp]))
cartoonif_vals = cartoonif_vals.transpose()
name = imclass + 'cartoonifcation_values.csv'
#cartoonif_vals.to_csv(name, index=False, header=("fn", "mnorm", "mnorm_perpixel", "znorm", "znorm_perpixel"))
print(cartoonif_vals.head())

### Inspect cartoonizaton results
---

In [ ]:
%cd ../
%cd image_data

# Plot parameters
kwargs = dict(alpha=0.5, bins=15)
fig, (a, b, c, d) = plt.subplots(4, figsize=(10, 10), sharey=True, constrained_layout=True)
fig.suptitle('Image differences after cartoonization (n=500 imgs)')

# Manhattan norm values
bins, counts = np.histogram(mnorms)
a.hist(mnorms, color='y', label='True Det', **kwargs)
a.set_title("Manhattan norm");

# Zero norm values
bins, counts = np.histogram(znorms)
c.hist(znorms, color='y', label='True Det', **kwargs)
c.set_title("Zero norm");


# Manhattan norm values per pixel
bins, counts = np.histogram(mnorms_pp)
b.hist(mnorms_pp, color='y', label='True Det', **kwargs)
b.set_title("Manhattan norm per pixel");

# Zero norm values per pixel
bins, counts = np.histogram(znorms_pp)
d.hist(znorms_pp, color='y', label='True Det', **kwargs)
d.set_title("Zero norm per pixel");

# TO DO: 
figname = 'phylo_cartoonization_hists' + '.png'
fig.savefig(figname)